# Discrete Derham complex

The basics of the 3d Derham diagram are explained in the [struphy documentation](https://struphy.pages.mpcdf.de/struphy/sections/discretization.html).

![hi](../doc/pics/derham_complex.png)

The discrete complex in the above diagram (lower row) is loaded via the **Derham class**: 

In [ ]:
import yaml
from mpi4py import MPI

import struphy
from struphy.psydac_api.psydac_derham import Derham

# use default input parameter file
path = struphy.__path__[0]
with open(path + '/io/inp/parameters.yml') as file:
    params = yaml.load(file, Loader=yaml.FullLoader)

Nel      = params['grid']['Nel']        # Number of grid cells
Nel[-1]  = 16 # use less points in third direction
p        = params['grid']['p']          # spline degrees
spl_kind = params['grid']['spl_kind']   # Spline types (clamped vs. periodic)

comm = MPI.COMM_WORLD
derham = Derham(Nel, p, spl_kind, comm=comm)

*Task 1:* Find out the number of elements, spline degrees and boundary conditions of the 1d spline spaces building up the tensor product bases.

In [ ]:
derham.Nel

In [ ]:
derham.p

In [ ]:
derham.spl_kind

*Task 2:* Create an element of each discrete space $V_h^n$ and initialize it (resp. its first component only) with a sinusoidal function of mode number 2 and amplitude 0.5 in the third direction.

In [ ]:
from struphy.psydac_api.fields import Field

p0 = Field('pressure', 'H1', derham)
E1 = Field('e_field', 'Hcurl', derham)
B2 = Field('b_field', 'Hdiv', derham)
n3 = Field('density', 'L2', derham)

init_params = {'type': 'ModesSin',  
               'ModesSin': {'coords': 'logical',
                            'comps': {'pressure': [True]},
                            'ls': [0], 
                            'ms': [0], 
                            'ns': [2], 
                            'amps': [.5]}}
p0.initialize_coeffs(init_params)

init_params['ModesSin']['comps'] = {'e_field': [True, False, False]}
E1.initialize_coeffs(init_params)

init_params['ModesSin']['comps'] = {'b_field': [True, False, False]}
B2.initialize_coeffs(init_params)

init_params['ModesSin']['comps'] = {'density': [True]}
n3.initialize_coeffs(init_params)

*Task 3:* Evaluate the 0-form, squeeze output and plot the result alongside the continuous sinusoidal function.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# sinusoidal
fun = lambda x, y, z : .5*np.sin(2*2*np.pi*z)

# evaluation points
eta1 = 0
eta2 = 0
eta3 = np.linspace(0., 1., 50)

# evaluate 0-form
p0_vals = p0(eta1, eta2, eta3, squeeze_output=True)
print('The evaluation output is a numpy array (squeezed in this case):\n')
print(p0_vals)

# plot
plt.plot(eta3, fun(0, 0, eta3), label='analytic')
plt.plot(eta3, p0_vals, 'o', label='0-form')
plt.legend()

*Task 4:* Evaluate the 1-form, squeeze output and plot the result alongside the continuous sinusoidal function.

In [ ]:
# evaluate 1-form
E1_vals = E1(eta1, eta2, eta3, squeeze_output=True)
print('The evaluation output is a list of numpy arrays (squeezed in this case):\n')
print(E1_vals)

# plot
plt.plot(eta3, fun(0, 0, eta3), label='analytic')
plt.plot(eta3, E1_vals[0], 'o', label='1-form 1')
plt.plot(eta3, E1_vals[1], 'x', label='1-form 2')
plt.plot(eta3, E1_vals[2], ':', label='1-form 3')
plt.legend()

*Task 5*: Project the sinusoidal function into $V_h^0$. Also, project its gradient into $V_h^1$.

In [ ]:
fun_h = derham.P['0'](fun)
print('Result of P0 projection is a StencilVector:', type(fun_h))


dx_fun = lambda x, y, z : 0*z
dy_fun = lambda x, y, z : 0*z
dz_fun = lambda x, y, z : 2*2*np.pi*.5*np.cos(2*2*np.pi*z)
grad_h = derham.P['1']((dx_fun, dy_fun, dz_fun))
print('Result of P1 projection is a BlockVector:', type(grad_h))

*Task 6:* Get the FE coefficients of the projected function and apply the discrete gradient operator.

In [ ]:
grad_fun_h = derham.grad.dot(fun_h)
print('Coefficients in V1 are a BlockVector:', type(grad_fun_h))

*Task 7:* Check the commuting property.

In [ ]:
assert np.allclose(grad_fun_h[0].toarray(), grad_fun_h[0].toarray())
assert np.allclose(grad_fun_h[1].toarray(), grad_fun_h[1].toarray())
assert np.allclose(grad_fun_h[2].toarray(), grad_fun_h[2].toarray())

### Other Derham object properties:

In [ ]:
# Derham discretization attributes
print(f'derham.bc: {Derham.bc.__doc__} {derham.bc}\n')
print(f'derham.quad_order: {Derham.quad_order.__doc__} {derham.quad_order}\n')
print(f'derham.nq_pr: {Derham.nq_pr.__doc__} {derham.nq_pr}')

In [ ]:
# Derham misc attributes
print(f'derham.breaks: {Derham.breaks.__doc__} {derham.breaks}\n')

In [ ]:
# Derham global spline index attributes
print(f'derham.indN: {Derham.indN.__doc__} {derham.indN}\n')
print(f'derham.indD: {Derham.indD.__doc__} {derham.indD}')

In [ ]:
# Derham parallel attributes
print(f'derham.comm: {Derham.comm.__doc__} {derham.comm}\n')
print(f'derham.domain_array: {Derham.domain_array.__doc__} {derham.domain_array}\n')
print(f'derham.index_array_N: {Derham.index_array_N.__doc__} {derham.index_array_N}\n')
print(f'derham.index_array_D: {Derham.index_array_D.__doc__} {derham.index_array_D}\n')
print(f'derham.neighbours: {Derham.neighbours.__doc__} {derham.neighbours}')

### All these operations also work in parallel !